⚙️ Step 1 – Install and import

In [1]:
!pip install groq --quiet
import os
from groq import Groq
import json


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 135.8/135.8 kB 7.1 MB/s eta 0:00:00


🔐 Step 2 – Set your API key

In [2]:
os.environ["GROQ_API_KEY"] = "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"
client = Groq(api_key=os.environ["GROQ_API_KEY"])


🧩 Step 3 – Prompt builder

In [17]:
def build_prompt(ticker, df):
    recent = df.tail(30)[['Date', 'Close']].copy()

    # More robust way to format dates and closes, avoiding iterrows() quirks
    formatted_dates = recent['Date'].dt.strftime('%d-%b-%Y')
    formatted_closes = recent['Close'].apply(lambda x: f'₹{x:.2f}')

    data_summary = "\n".join([
        f"{date_str}: {close_str}"
        for date_str, close_str in zip(formatted_dates, formatted_closes)
    ])

    prompt = f"""
YouAre a financial analyst AI. Analyze the following 30-day stock closing prices for {ticker}.

Data:
{data_summary}

Please respond strictly in JSON with keys:
"summary" → 2-3 sentence overview of trend,
"risk" → Low / Medium / High,
"suggestion" → Buy / Hold / Sell.

Example format:
{{
  "summary": "...",
  "risk": "Medium",
  "suggestion": "Hold"
}}
    """
    return prompt.strip()


🚀 Step 4 – Send request to Groq API

In [41]:
def analyze_with_llm(ticker, df):
    prompt = build_prompt(ticker, df)

    # Use the correct, currently supported model
    response = client.chat.completions.create(
        model="llama-3.1-8b-instant",
        messages=[{"role": "user", "content": prompt}],
        temperature=0.3,
        max_tokens=300,
    )

    raw = response.choices[0].message.content.strip()
    print("Raw LLM Response:\n", raw)

    # Try to parse JSON output safely
    try:
        parsed = json.loads(raw)
    except json.JSONDecodeError:
        parsed = {"summary": raw, "risk": "Unknown", "suggestion": "Hold"}

    return parsed


🧪 Step 5 – Run test

In [42]:
import pandas as pd
import yfinance as yf
import matplotlib.pyplot as plt # Needed for plot_prediction
import json

# 1️⃣ Define ticker and fetch data safely
ticker = "RELIANCE.NS"
# Use yf.Ticker().history() for potentially more robust single-ticker data fetching
# It returns Date as index and standard columns.
# auto_adjust=False will ensure both 'Close' and 'Adj Close' are present
stock_data = yf.Ticker(ticker)
df = stock_data.history(start="2020-01-01", end="2023-01-01", auto_adjust=False)

# 2️⃣ Robust column handling
# The history() method usually returns a clean DataFrame with Date as index.
# Flatten MultiIndex columns if they exist. (Defensive, but history() usually avoids it).
if isinstance(df.columns, pd.MultiIndex):
    # This takes the last level of the MultiIndex, which usually contains the actual column names (Open, Close, etc.)
    df.columns = df.columns.get_level_values(-1)

# Reset the index. This will turn the 'Date' index into a regular column named 'Date'.
df.reset_index(inplace=True)

# Identify and standardize 'Date' and 'Close' explicitly.
# We now expect 'Date' to be a column named 'Date' after reset_index().
# We want 'Adj Close' if available, otherwise 'Close'.
date_col = 'Date' # Expected column name after reset_index()

close_col = None
if 'Adj Close' in df.columns:
    close_col = 'Adj Close'
elif 'Close' in df.columns:
    close_col = 'Close'

if close_col is None:
    # This should not happen with auto_adjust=False on history() if data is available.
    raise ValueError(f"Could not find 'Close' or 'Adj Close' related columns. Current columns: {df.columns.tolist()}")

# Create a new DataFrame with just 'Date' and 'Close' and explicitly rename them.
df = df[[date_col, close_col]].copy()
df.columns = ['Date', 'Close'] # Explicitly set column names to 'Date' and 'Close'

# Ensure 'Date' column is a datetime object.
df['Date'] = pd.to_datetime(df['Date'])

print("✅ Clean DataFrame Columns:", df.columns.tolist())
print(df.head())

# 3️⃣ Call your LLM analysis function (make sure it’s already defined)
llm_result = analyze_with_llm(ticker, df)

# 4️⃣ Print structured output
print("\n🧠 LLM Analysis:")
print(json.dumps(llm_result, indent=2))


✅ Clean DataFrame Columns: ['Date', 'Close']
                       Date       Close
0 2020-01-01 00:00:00+05:30  675.324158
1 2020-01-02 00:00:00+05:30  686.821228
2 2020-01-03 00:00:00+05:30  687.648804
3 2020-01-06 00:00:00+05:30  671.700745
4 2020-01-07 00:00:00+05:30  682.034546
Raw LLM Response:
 {
  "summary": "The stock price of RELIANCE.NS showed an overall upward trend from ₹1164.48 on 21-Nov-2022 to ₹1246.85 on 30-Nov-2022, but then experienced a decline until ₹1142.25 on 23-Dec-2022. The price then recovered slightly, ending at ₹1162.79 on 30-Dec-2022. This indicates a volatile market with fluctuations in the stock's value.",
  "risk": "Medium",
  "suggestion": "Hold"
}

🧠 LLM Analysis:
{
  "summary": "The stock price of RELIANCE.NS showed an overall upward trend from \u20b91164.48 on 21-Nov-2022 to \u20b91246.85 on 30-Nov-2022, but then experienced a decline until \u20b91142.25 on 23-Dec-2022. The price then recovered slightly, ending at \u20b91162.79 on 30-Dec-2022. This 